### [YouTube lecture](https://www.youtube.com/watch?v=uAlp2VuZZPY&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=57)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('pyspark_test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 04:11:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 04:11:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
year = '2021'
df_green = spark.read.parquet(f'../../data/pq/green/*/*')
df_yellow = spark.read.parquet(f'../../data/pq/yellow/*/*')

In [4]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [6]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [7]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [8]:
common_columns = []

yellow_columns = set(df_yellow.columns)

for column in df_green.columns:
    if column in yellow_columns:
        common_columns.append(column)

common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [9]:
# create a new column in each table specifying the service color (green or yellow)

df_green_serv = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

df_green_serv.show(1, vertical=True)

-RECORD 0------------------------------------
 VendorID              | 2                   
 pickup_datetime       | 2020-01-23 13:10:15 
 dropoff_datetime      | 2020-01-23 13:38:16 
 store_and_fwd_flag    | N                   
 RatecodeID            | 1                   
 PULocationID          | 74                  
 DOLocationID          | 130                 
 passenger_count       | 1                   
 trip_distance         | 12.77               
 fare_amount           | 36.0                
 extra                 | 0.0                 
 mta_tax               | 0.5                 
 tip_amount            | 2.05                
 tolls_amount          | 6.12                
 improvement_surcharge | 0.3                 
 total_amount          | 44.97               
 payment_type          | 1                   
 congestion_surcharge  | 0.0                 
 service_type          | green               
only showing top 1 row



In [10]:
df_yellow_serv = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

df_yellow_serv.show(1, vertical=True)

-RECORD 0------------------------------------
 VendorID              | 2                   
 pickup_datetime       | 2020-01-06 09:18:38 
 dropoff_datetime      | 2020-01-06 09:33:56 
 store_and_fwd_flag    | N                   
 RatecodeID            | 1                   
 PULocationID          | 263                 
 DOLocationID          | 233                 
 passenger_count       | 1                   
 trip_distance         | 3.03                
 fare_amount           | 12.0                
 extra                 | 0.0                 
 mta_tax               | 0.5                 
 tip_amount            | 1.5                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 16.8                
 payment_type          | 1                   
 congestion_surcharge  | 2.5                 
 service_type          | yellow              
only showing top 1 row



In [11]:
df_trips_data = df_green_serv.unionAll(df_yellow_serv)
df_trips_data.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- service_type: string (nullable = false)



In [12]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [13]:
# Tell spark that this df_trips_data is a table

df_trips_data.registerTempTable('trips_data')

/home/ramazan/spark/spark-3.5.0-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [14]:
spark.sql("""
    SELECT service_type, COUNT(service_type) AS count FROM trips_data
    GROUP BY service_type
""").show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [19]:
df_result = spark.sql("""
SELECT 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 
    
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

FROM trips_data
GROUP BY revenue_zone, revenue_month, service_type
""")

In [20]:
df_result.count()

9895

In [22]:
df_result.write.parquet(f'../../data/report/revenue/combined', mode="overwrite")